In [1]:
import itertools
from functools import reduce
from itertools import product

import operator

K = Fields()

q = 2**61 - 1
Fq = GF(q)
x = polygen(Fq, 'x')
Fq2.<u> = Fq.extension(x^2+1)
log_n = 4
n = 2**log_n
v = vector(Fq, [Fq.random_element() for _ in range(n)])

def get_bit(n, k):
    return (n >> k) & 1

def int_to_bits(n, bit_length=None):
    if bit_length is None:
        bit_length = n.bit_length() if n != 0 else 1
    return [(n >> i) & 1 for i in range(bit_length)]

def multilinear_extension(v, F, var_names=None):
    n = len(v)
    m = n.bit_length() - 1
    if 2 ** m != n:
        raise ValueError("Vector length must be a power of 2.")
    
    if var_names is None:
        var_names = [f'X{i}' for i in range(1, m+1)]
    R = PolynomialRing(F, var_names)
    X = R.gens()
    p = R.zero()
    
    for i in range(n):
        term = F(v[i]) 
        bits = int_to_bits(i,m) 
        
        for j in range(m):
            if bits[j] == 1:
                term *= X[j]
            else:
                term *= (1 - X[j])
        p += term
    return p

v_tilde = multilinear_extension(v, Fq)

for i, t in enumerate(v):
  assert t == v_tilde([j for j in int_to_bits(i, log_n)])


def multilinear_matrix_extension(M, F, var_names=None):
    d, n = M.dimensions()
    total_elements = d * n
    
    m = total_elements.bit_length() - 1
    if 2^m != total_elements:
        raise ValueError("Matrix dimensions must multiply to a power of 2 (d * n = 2^m).")
    
    # Flatten the matrix into a vector (row-major order)
    v = M.list()
    
    if var_names is None:
        var_names = [f'X{i}' for i in range(1, m+1)]
    R = PolynomialRing(F, var_names)
    X = R.gens()
    
    p = R.zero()
    for i in range(total_elements):
        term = F(v[i])  
        bits = int_to_bits(i,m) 
        
        for j in range(m):
            if bits[j] == 1:
                term *= X[j]
            else:
                term *= (1 - X[j])
        
        p += term
    
    return p

M = Matrix(Fq, [[ Fq.random_element() for _ in range(n)] for _ in range(n)])
M_tilde = multilinear_matrix_extension(M, Fq)

for i, row in enumerate(M):
  for j, e in enumerate(row):
    assert M[i][j] == M_tilde([y for y in int_to_bits(j, log_n)]+[x for x in int_to_bits(i, log_n)])

## TENSOR PRODUCT TEST
# Build multi-linear basis polynomials
var_names = [f'X{i}' for i in range(log_n)]
R = PolynomialRing(Fq2, var_names)
X = R.gens()
print(X)
# print([X[i] for i in range(log_n)])
basis = []
for i in range(n):
    bits = [(i >> j) & 1 for j in range(log_n)]
    poly = 1
    for b, x in zip(bits, [X[i] for i in range(log_n)]):
        poly *= x if b else (1 - x)
    basis.append(poly)

# print(basis)

tilde_f_r = sum(v[i] * basis[i] for i in range(n))
v_tilde2 = multilinear_extension(v, Fq2)
for i, t in enumerate(v):
  params = [j for j in int_to_bits(i, log_n)]
  assert tilde_f_r(params) == v_tilde(params)


## TENSOR PRODUCT BY PAPER DEFINITION
## compute at a random point r
r = vector(Fq2, [Fq2.random_element() for _ in range(log_n)])
# r_tilde = [r[0], 1 - r[0]]
# for i in range(1, log_n):
#     r_temp = []
#     for ri in r_tilde:
#         for rj in [r[i], 1 - r[i]]:
#             r_temp.append(ri * rj)
#     r_tilde = r_temp
# assert len(r_tilde) == n
# print(r_tilde)

def compute_ml_extension_tensor(v, r):
    n = len(v)
    assert len(r) == n.bit_length() - 1, "Length of r must match log_n"
    r_tilde = []
    # multilinear_extension and int_to_bits treat r as corresponding to the least significant bit of the index i.
    for i in range(n):
        term_product = Fq2(1)
        bits = int_to_bits(i, log_n)
        
        for j in range(log_n):
            if bits[j] == 1:
                term_product *= r[j]
            else:
                term_product *= (1 - r[j])
        r_tilde.append(term_product)

    return sum(v[i] * r_tilde[i] for i in range(n)), r_tilde

tilde_f_r, r_tilde = compute_ml_extension_tensor(v, r)
r_tuple = tuple(r) 

# Evaluate the multilinear extension at point r
v_tilde_at_r = v_tilde(*r_tuple) # Unpack the tuple into arguments
assert v_tilde_at_r == tilde_f_r


(X0, X1, X2, X3)


In [2]:
# 2.3 Rings and Modules
eta = 81
d = 54
q = 2**61 - 1
Fq = GF(q)
x = polygen(Fq, 'x')
Phi = x ** 54 + x ** 27 + 1
Rq.<u> = Fq.extension(Phi)
kappa = 16
## m = 2 ** 22 # TOO MUCH MEMORY
#m = 2 ** 8 # for testing purposes
#M = random_matrix(Rq, kappa, m)
a = Rq.random_element()

def cf(a):
    return a.list()

def cf_inv(a):
    return Rq(a)

def shift_matrix(Fq, Phi, d):
    F = matrix(Fq, d, d)
    c = cf(Phi)
    for i in range(d):
        for j in range(d):
            if i - 1 == j:
                F[i, j] = 1
            if j == d - 1:
                F[i, j] = -c[i]
    return F

def rot(a, Phi, Fq):
    cf_a = vector(Fq, a if isinstance(a, list) else cf(a))
    d = len(cf_a)
    F = shift_matrix(Fq, Phi, d)

    columns = [cf_a]
    F_power = F  # Start with F^1
    for _ in range(1, d):
        current = F_power * cf_a
        columns.append(current)
        F_power = F_power * F  # Compute next power incrementally
    rows = list(zip(*columns))
    M = matrix(Fq, len(cf_a), len(cf_a), sum([list(row) for row in rows], []))

    return M    # print([(F**i) * cf_a for i in range(1, d)])
    
# print(shift_matrix(Fq, Phi, d))
rot_a = rot(a, Phi, Fq)
b = Rq.random_element()
assert rot_a * vector(Fq, cf(b)) == vector(Fq,cf(a*b))

# F7 = GF(7)

# y = polygen(F7, 'y')
# Phi_test = y ** 3 + 1
# R7.<u> = F7.extension(Phi_test)

# a = 4*u**2 + 2*u + 3
# rot_a = rot(a, Phi_test, F7)
# print(rot_a)
# b = u**2 + 1
# print("cf(a*b)", cf(a*b))
# print("rot_a * vector(F7, cf(b)", rot_a * vector(F7, cf(b)))
# assert rot_a * vector(F7, cf(b)) == vector(F7, cf(a*b))

# 3.2 Neo’s solution, part-1: A matrix commitment scheme

def embed_Fq_to_Rq(z_i, d):
    """
    Embed a field element z_i ∈ Fq into Rq by using its bits
    as coefficients of a polynomial in Rq.
    """
    z_int = int(z_i)
    bits = [((z_int >> i) & 1) for i in range(d)]
    poly = sum(Fq(bits[i]) * u^i for i in range(len(bits)))
    return Rq(poly)

z_i = Fq.random_element()
z_i_embedded = embed_Fq_to_Rq(z_i, d)

print(f"Original Fq element: {z_i}")
print(f"Embedded Rq element: {z_i_embedded}")


Original Fq element: 1774342155030026838
Embedded Rq element: u^52 + u^51 + u^50 + u^49 + u^48 + u^47 + u^45 + u^44 + u^43 + u^41 + u^39 + u^38 + u^35 + u^33 + u^26 + u^23 + u^20 + u^15 + u^14 + u^12 + u^11 + u^9 + u^6 + u^4 + u^2 + u


In [3]:
from random import randint


def decomp_b(z, d=None):
    F = z.base_ring()
    m = len(z)
    b = 2

    # Determine degree if not provided
    if d is None:
        # Find smallest d such that b^d > max(z)
        max_val = max([abs(int(zi)) for zi in z])
        d = 1
        while b**d <= max_val:
            d += 1

    # Initialize the result matrix
    # Z = matrix(F, d, m)

    # Perform b-ary decomposition
    cols = []
    for j in range(m):
        value = int(z[j])
        bits = [((value >> i) & 1) for i in range(d)]
        cols.append(bits)
    rows = list(zip(*cols))
    Z = matrix(F, d, m, sum([list(row) for row in rows], []))

    return Z


z_1 = Fq(123)
z_2 = Fq(567)
z_3 = Fq(890)
m = 3
z_1_embedded = embed_Fq_to_Rq(z_1, d)
z_2_embedded = embed_Fq_to_Rq(z_2, d)
z_3_embedded = embed_Fq_to_Rq(z_3, d)
z_vec = vector(Fq, [z_1, z_2, z_3])
print(z_vec)
Z = decomp_b(z_vec, d)
print("len(cf(z_1_embedded))", len(cf(z_1_embedded)))
print("len([Z[i, 0] for i in range(d)])", len([Z[i, 0] for i in range(d)]))
assert cf(z_1_embedded) == [Z[i, 0] for i in range(d)]
assert cf(z_2_embedded) == [Z[i, 1] for i in range(d)]
assert cf(z_3_embedded) == [Z[i, 2] for i in range(d)]

print("check sum", [Fq(sum([2**i * Z[i, col] for i in range(d)])) for col in range(m)])
assert list(z_vec) == [
    Fq(sum([2**i * Z[i, col] for i in range(d)])) for col in range(m)
]


def split_b(Z, k=None):
    F = Z.base_ring()
    d, m = Z.dimensions()
    b = 2

    # Determine depth if not provided
    if k is None:
        # Find smallest k such that b^k > max(Z)
        max_val = max([abs(int(Z[i, j])) for i in range(d) for j in range(m)])
        k = 1
        while b**k <= max_val:
            k += 1
    # Initialize the result matrices
    result = [matrix(F, d, m) for _ in range(k)]

    # Perform b-ary decomposition
    for i in range(d):
        for j in range(m):
            value = int(Z[i, j])
            bits = [((value >> i) & 1) for i in range(k)]
            for l in range(k):
                result[l][i, j] = F(bits[l])
    return result


max_val = 2**20  # low norm
test = matrix(Fq, d, m)
for i in range(d):
    for j in range(m):
        test[i, j] = Fq(randint(0, max_val - 1))
Z = split_b(test)
Z_restored = matrix(Fq, d, m)
for i, z in enumerate(Z):
    Z_restored += (2**i) * z

print(Z_restored)
assert test == Z_restored

(123, 567, 890)
len(cf(z_1_embedded)) 54
len([Z[i, 0] for i in range(d)]) 54
check sum [123, 567, 890]
[ 164645  480111  958586]
[ 635171  558860  337444]
[ 295089  380635 1015750]
[ 733023  158630  146624]
[ 807485  958610  378366]
[ 484951  616452  100412]
[ 733709  243366  594876]
[  35267  817185  218770]
[  87225   47253  772845]
[ 915215  674601  935841]
[  13760   77720 1034934]
[ 920871  160158   42877]
[  55069   84716  526463]
[  39895 1032660  594912]
[  91759  378378  351845]
[1009118   65633  135757]
[ 835473  303990  333307]
[ 121650  302961  560651]
[ 835299 1045950  857988]
[ 610748  619202  973692]
[ 614166  406940  294833]
[ 896200  162578  944217]
[ 597131  454719  912150]
[  18983    6897 1019498]
[ 481596  652436  987075]
[ 581946  914257  626210]
[ 358053  595800  548726]
[ 444309  482425  639192]
[ 845256  187836  231625]
[ 588486  187482  536278]
[ 362718  338649  609026]
[ 670530  811078 1002910]
[ 503192  282935  750086]
[ 866835  728275 1026618]
[ 853186  885

In [4]:
import json

with open("../mimc_r1cs.json", "r") as f:
    r1cs = json.load(f)

# "Without loss of generality, assume that m = n and n, d · n are both powers of two"

z = vector(Fq, [int(w, base=16) for w in r1cs["witness"]])
assert all([x < 2**d for x in z])
print(z)
m = len(z)
if m & (m - 1) != 0:
    next_pow2 = 1 << int(m - 1).bit_length()
    print("next_pow2", next_pow2)
    z = vector(Fq, list(z) + [0] * (next_pow2 - m))
    m = len(z)
    print("m", m)


Z = decomp_b(z, d)

p_vec = vector(Fq, [2**i for i in range(Z.nrows())])
z_reconstructed_from_Z = Z.transpose() * p_vec
print("z_reconstructed_from_Z", z_reconstructed_from_Z)
# Ensure the witness z is perfectly reconstructed from the Z matrix.
assert (
    z == z_reconstructed_from_Z
), "z is not correctly reconstructed from Z. Check decomp_b."


print(Z.dimensions()[0])
assert d == Z.dimensions()[0]
m = Z.dimensions()[1]
print("Z.dimensions()", Z.dimensions())
for i in range(Z.nrows()):
    for j in range(Z.ncols()):
        assert Z[i, j] != 0 or Z[i, j] != 1

# In particular, each column of a low-norm matrix gets mapped as the coefficients of a single ring element.
z_prime = vector(Rq, [cf_inv(list(Z.column(j))) for j in range(Z.ncols())])
print("z_prime", z_prime)
print(type(z_prime))
pp = random_matrix(Rq, kappa, m)
print("pp.dimensions()", pp.dimensions())
print("type(pp)", type(pp))
print("z_prime", type(z_prime))
c = pp * z_prime

rows = []
for x in c:
    rows.append(cf(x))
C_commitment = matrix(Fq, kappa, d, sum([list(row) for row in rows], [])).transpose()
print("C_commitment.dimensions()", C_commitment.dimensions())

(1, 16962, 291, 297666009, 5135631653277)
next_pow2 8
m 8
z_reconstructed_from_Z (1, 16962, 291, 297666009, 5135631653277, 0, 0, 0)
54
Z.dimensions() (54, 8)
z_prime (1, u^14 + u^9 + u^6 + u, u^8 + u^5 + u + 1, u^28 + u^24 + u^23 + u^21 + u^20 + u^19 + u^18 + u^17 + u^10 + u^8 + u^7 + u^6 + u^4 + u^3 + 1, u^42 + u^39 + u^37 + u^35 + u^33 + u^32 + u^31 + u^29 + u^28 + u^27 + u^25 + u^24 + u^23 + u^12 + u^11 + u^8 + u^7 + u^4 + u^3 + u^2 + 1, 0, 0, 0)
<class 'sage.modules.free_module_element.FreeModuleElement_generic_dense'>
pp.dimensions() (16, 8)
type(pp) <class 'sage.matrix.matrix_generic_dense.Matrix_generic_dense'>
z_prime <class 'sage.modules.free_module_element.FreeModuleElement_generic_dense'>
C_commitment.dimensions() (54, 16)


In [5]:
# CCS Reduction (ΠCCS).
# This reduction takes a new MCS(b,L) instance (your new computation) and k-1 existing ME(b,L) claims, reducing them to k new ME(b,L) claims
k = 12
T = 216
### Definition 17 (Matrix constraint system relation).


# "Without loss of generality, assume that m = n and n, d · m are both powers of two"
def extend_matrix(M, size):
    rows, cols = M.dimensions()

    # Check that size is sufficient for the matrix
    if size < rows or size < cols:
        raise ValueError(
            f"Target size {size} is too small for matrix with dimensions {rows}x{cols}"
        )

    # If the matrix already has the target dimensions, return it
    if rows == cols == size:
        return M

    # Create a new square matrix of zeros with the target size
    result = matrix(M.base_ring(), size, size)

    # Copy the original matrix into the top-left corner
    for i in range(rows):
        for j in range(cols):
            result[i, j] = M[i, j]

    return result


# check r1cs constraint
A = matrix(Fq, r1cs["A"])
B = matrix(Fq, r1cs["B"])
C = matrix(Fq, r1cs["C"])
x = vector(Fq, [int(w, base=16) for w in r1cs["witness"]])
print([int(w, base=16) for w in r1cs["witness"]])
print(vector(Fq, [int(w, base=16) for w in r1cs["witness"]]))
M0x = A * x
M1x = B * x
M2x = C * x
print(M0x)
print(M1x)
print(M2x)
hadamard = vector(Fq, [u * v for u, v in zip(M0x, M1x)])
print(hadamard)
assert hadamard == M2x, f"R1CS failed: {hadamard} != {M2x}"
print("R1CS constraints are satisfied!")

M0 = extend_matrix(A, m)
M1 = extend_matrix(B, m)
M2 = extend_matrix(C, m)


print("M0.dimensions()", M0.dimensions())
print("M1.dimensions()", M1.dimensions())
print("M2.dimensions()", M2.dimensions())
print("len(z)", len(z))

M0z = M0 * z
M1z = M1 * z
M2z = M2 * z
print("len(M0z)", len(M0z))


M0z_tilde = multilinear_extension(M0z, Fq2)
M1z_tilde = multilinear_extension(M1z, Fq2)
M2z_tilde = multilinear_extension(M2z, Fq2)
# Check if R1CS constraint holds: (A·z) ⊙ (B·z) = C·z
hadamard_product = vector(Fq, [M0z[i] * M1z[i] for i in range(len(M0z))])
print(hadamard_product)
print(M2z)
# Check if the constraint is satisfied
constraint_satisfied = hadamard_product == M2z
print("constraint_satisfied", constraint_satisfied)
F = M0z_tilde * M1z_tilde - M2z_tilde
print(F)
bit_combinations = list(product([0, 1], repeat=3))

# Check that F is vanishing over the hypercube
results = []
for bits in bit_combinations:
    assert F(*bits) == 0

[1, 16962, 291, 297666009, 5135631653277]
(1, 16962, 291, 297666009, 5135631653277)
(17253, 297666009)
(17253, 17253)
(297666009, 5135631653277)
(297666009, 5135631653277)
R1CS constraints are satisfied!
M0.dimensions() (8, 8)
M1.dimensions() (8, 8)
M2.dimensions() (8, 8)
len(z) 8
len(M0z) 8
(297666009, 5135631653277, 0, 0, 0, 0, 0, 0)
(297666009, 5135631653277, 0, 0, 0, 0, 0, 0)
constraint_satisfied True
5135333987268*X1*X2^2*X3^2 + 2305832738545719415*X1*X2^2*X3 + 2305832738545719415*X1*X2*X3^2 + 297666009*X2^2*X3^2 + 5135333987268*X1*X2^2 + 15406001961804*X1*X2*X3 + 2305843008618361933*X2^2*X3 + 5135333987268*X1*X3^2 + 2305843008618361933*X2*X3^2 + 2305837873879706683*X1*X2 + 297666009*X2^2 + 2305837873879706683*X1*X3 + 892998027*X2*X3 + 297666009*X3^2 + 2305843008916027942*X2 + 2305843008916027942*X3


In [22]:
# 3.3 Neo's solution, part-2: linear homomorphism for folding multilinear evaluation claims
# Test Procedure
# 1. **Start with a known correct witness $z$** for your R1CS.
# 2. **Compute $M_j z \in \mathbb{F}^m$** for your constraint matrix $M_j \in \mathbb{F}^{m \times n}$.
# 3. **Evaluate the multilinear extension** of $M_j z$ at point $r$:

#    $$
#    y_1 := \tilde{M_j z}(r) = \langle M_j z, \hat{r} \rangle
#    $$

# 4. **Decompose $z \rightarrow Z \in \mathbb{F}^{d \times n}$** using `Decomp_b(z)`.

# 5. **Compute** $Z M_j^\top \in \mathbb{F}^{d \times m}$

# 6. **Multiply with $\hat{r}$**:

#    $$
#    y_2 := (Z M_j^\top) \cdot \hat{r}
#    $$

# 7. **Recombine bits** (i.e., use the base $b$ powers):

#    $$
#    \boxed{
#    y := \sum_{i=0}^{d-1} b^i \cdot y_2^{(i)} \quad \text{(dot product per slice)}
#    }
#    $$

# 8. Then:

#    $$
#    \boxed{ y = y_1 }
#    \quad \text{(This is your invariant)}
#    $$

n = M0.ncols()
log_n = n.bit_length() - 1
r = vector(Fq2, [Fq2.random_element() for _ in range(log_n)])
print("len(r)", len(r))
y1, r_hat = compute_ml_extension_tensor(M0z, r)
print("M0z_tilde", M0z_tilde)
assert y1 == M0z_tilde(*tuple(r))
ZM0t = Z * M0.transpose()
y2 = ZM0t * vector(Fq2, r_hat)
y = sum(
    [2**i * y2[i] for i in range(ZM0t.nrows())]
)  # Recombine bits using base b powers
print("y1", y1)
print("y", y)

assert y == y1

len(r) 3
M0z_tilde 297648756*X1*X2*X3 + 2305843008916045195*X1*X2 + 2305843008916045195*X1*X3 + 17253*X2*X3 + 297648756*X1 + 2305843009213676698*X2 + 2305843009213676698*X3 + 17253
y1 892165996054165059*u + 1460553523351139737
y 892165996054165059*u + 1460553523351139737


In [ ]:
## Definition 18 (Matrix evaluation relation).


class S_R1CS:
    def __init__(self, Fq2, A, B, C, z):
        self.z = z
        # "Without loss of generality, assume that m = n and n, d · n are both powers of two"
        m = len(self.z)
        if m & (m - 1) != 0:
            next_pow2 = 1 << int(m - 1).bit_length()
            self.z = vector(self.Fq, list(self.z) + [0] * (next_pow2 - m))
            m = len(self.z)
        self.m = m
        self.log_m = self.m.bit_length() - 1

        self.M = [extend_matrix(A, m), extend_matrix(B, m), extend_matrix(C, m)]
        self.M_tilde = [multilinear_matrix_extension(m, Fq2) for m in self.M]

        self.Mz = [m * self.z for m in self.M]
        self.Mz_tilde = [multilinear_extension(m, Fq2) for m in self.Mz]
        self.f = self.Mz_tilde[0] * self.Mz_tilde[1] - self.Mz_tilde[2]
        self.t = 3  # Number of matrices in the relation
        bit_combinations = list(product([0, 1], repeat=self.log_m))
        # Check that f is vanishing over the hypercube
        results = []
        for bits in bit_combinations:
            assert self.f(*bits) == 0


class MatrixEvaluation:

    def __init__(self, Fq, Rq, s, d, kappa, pp=None):
        self.s = s
        self.d = d
        self.kappa = kappa
        self.Fq = Fq
        self.Rq = Rq
        self.pp = pp

    def commit(self, s: S_R1CS, m_in=None):

        self.s = s

        if not m_in:
            self.m_in = m

        if not self.pp:
            self.pp = random_matrix(self.Rq, self.kappa, self.m)

        self.Z = decomp_b(self.s.z, d=self.d)
        self.X = self.Z[:, : self.m_in]
        assert all(self.Z.column(i) == self.X.column(i) for i in range(self.m_in))
        assert self.d == self.Z.dimensions()[0]
        p_vec = vector(Fq, [2**i for i in range(Z.nrows())])
        z_reconstructed_from_Z = Z.transpose() * p_vec
        # Ensure the witness z is perfectly reconstructed from the Z matrix.
        assert (
            z == z_reconstructed_from_Z
        ), "z is not correctly reconstructed from Z. Check decomp_b."

        for i in range(self.Z.nrows()):
            for j in range(self.Z.ncols()):
                assert self.Z[i, j] == 0 or self.Z[i, j] == 1

        # Each column of a low-norm matrix gets mapped as the coefficients of a single ring element.
        z_prime = vector(
            self.Rq, [cf_inv(list(self.Z.column(j))) for j in range(self.Z.ncols())]
        )

        self.c = self.pp * z_prime
        rows = []
        for x in self.c:
            rows.append(cf(x))
        self.C = matrix(
            self.Fq, self.kappa, self.d, sum([list(row) for row in rows], [])
        ).transpose()

        self.y = []
        log_n = m.bit_length() - 1
        self.r = vector(Fq2, [Fq2.random_element() for _ in range(log_n)])

        for i in range(3):
            y, r_hat = compute_ml_extension_tensor(self.s.M[i] * self.s.z, self.r)
            self.y.append(y)
            print("self.s.Mz_tilde[i]", self.s.Mz_tilde[i])
            print("len(self.r)", len(self.r))
            assert y == (self.s.Mz_tilde[i])(*tuple(self.r))


me = MatrixEvaluation(Fq, Rq, None, d, kappa, pp)
me.commit(S_R1CS(Fq2, A, B, C, z))
assert C_commitment == me.C

8
self.s.Mz_tilde[i] 297648756*X1*X2*X3 + 2305843008916045195*X1*X2 + 2305843008916045195*X1*X3 + 17253*X2*X3 + 297648756*X1 + 2305843009213676698*X2 + 2305843009213676698*X3 + 17253
len(self.r) 3
self.s.Mz_tilde[i] 17253*X2*X3 + 2305843009213676698*X2 + 2305843009213676698*X3 + 17253
len(self.r) 3
self.s.Mz_tilde[i] 5135333987268*X1*X2*X3 + 2305837873879706683*X1*X2 + 2305837873879706683*X1*X3 + 297666009*X2*X3 + 5135333987268*X1 + 2305843008916027942*X2 + 2305843008916027942*X3 + 297666009
len(self.r) 3
